# Gap up
**Truth Premise: When a day ends and a new day begins in life of options premium, its value depreciates because of time decay because it gets closer to expiry and the probability of its contract going as expected by the buyer gets lesser.**

In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
# DAILY_INVESTMENT = 1 * (10 ** 5) # 1 lakh
LOT_SIZE = 50
LOT_QTY = 10

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = all_dates
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

@cache
def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

@cache
def get_symbol_first_candle(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open, data.iloc[0].high, data.iloc[0].low, data.iloc[0].close

@cache
def get_first_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].close

@cache
def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 1), axis=1)

def get_nifty_price(d, t):
    data = ut.get_data(symbol=SYMBOL, date=d, interval=INTERVAL, exchange=EXCHANGE)
    try:
        return data.loc[data.index.time == t].iloc[0].open
    except IndexError:
        return pd.NA
    except AttributeError as e:
        return pd.NA

buy_time = dt.time(hour=15, minute=28)
sell_time = dt.time(hour=9, minute=16)

train_dates = train_dates.copy()

# train_dates["trade_day_before_expiry"] = train_dates.expiry - pd.Timedelta(days=day_before_expiry)
train_dates.loc[:, "nifty_at_0320"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), dt.time(hour=15, minute=20)), axis=1)
train_dates.loc[:, "td_nifty_at_0918"] = train_dates.apply(lambda r: get_nifty_price(r.name.date(), dt.time(hour=9, minute=15)), axis=1)
train_dates.loc[:, "td_nifty_at_0328"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), dt.time(hour=15, minute=29)), axis=1)
train_dates.loc[:, "td_nifty_diff"] = train_dates.td_nifty_at_0328 - train_dates.td_nifty_at_0918
train_dates.loc[:, "nifty_at_buy"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), buy_time), axis=1)
train_dates.loc[:, "nifty_at_sell"] = train_dates.apply(lambda r: get_nifty_price(r.name.date(), sell_time), axis=1)
train_dates.loc[:, "nifty_diff"] = train_dates.nifty_at_sell - train_dates.nifty_at_buy

train_dates = train_dates.loc[train_dates.nifty_at_0320.notna()]
train_dates["atm_strike"] = train_dates.apply(lambda trade: round(trade.nifty_at_0320 / 50) * 50, axis=1)

In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].close

def get_next_1000(strike, i):
    divider = 50
    reminder = strike % divider
    if reminder != 0:
        return divider * (strike // divider) - i * divider
    return divider * (strike // divider) - (i+1) * divider

last_strike = None
"""
Results Training dataset 2nd expiry
    Call PnL at 9:18 for strike: -10, Total: -759835.0, Per Day: -2140.3802816901407, Days: 355, Pc Mean: -2.14975530819984
Call PnL at 9:18 for strike: -9, Total: -312092.4999999999, Per Day: -857.3969780219777, Days: 364, Pc Mean: -0.8748627952745024
Call PnL at 9:18 for strike: -8, Total: 71502.49999999994, Per Day: 195.36202185792334, Days: 366, Pc Mean: 0.18297912502349303
Call PnL at 9:18 for strike: -7, Total: 506540.0, Per Day: 1380.2179836512262, Days: 367, Pc Mean: 1.369693395280383
Call PnL at 9:18 for strike: -6, Total: 641875.0, Per Day: 1744.2255434782608, Days: 368, Pc Mean: 1.743611133234208
Call PnL at 9:18 for strike: -5, Total: 755737.5, Per Day: 2053.634510869565, Days: 368, Pc Mean: 2.054257620026519
    Call PnL at 9:18 for strike: -4, Total: 826650.0, Per Day: 2246.3315217391305, Days: 368, Pc Mean: 2.2097529699576683
Call PnL at 9:18 for strike: -3, Total: 807337.5000000002, Per Day: 2193.8519021739135, Days: 368, Pc Mean: 2.1964092790268843
Call PnL at 9:18 for strike: -2, Total: 720057.5, Per Day: 1956.6779891304348, Days: 368, Pc Mean: 1.9887079193200083
Call PnL at 9:18 for strike: -1, Total: 641890.0000000001, Per Day: 1744.2663043478265, Days: 368, Pc Mean: 1.8311249840868105
Call PnL at 9:18 for strike: 0, Total: 577307.5000000002, Per Day: 1568.7703804347832, Days: 368, Pc Mean: 1.619612769168954
Call PnL at 9:18 for strike: 1, Total: 526917.5, Per Day: 1455.5732044198894, Days: 362, Pc Mean: 1.5759072833721137
Call PnL at 9:18 for strike: 2, Total: 480467.50000000006, Per Day: 1392.6594202898552, Days: 345, Pc Mean: 1.5418813730866745
Call PnL at 9:18 for strike: 3, Total: 411579.99999999994, Per Day: 1331.9741100323622, Days: 309, Pc Mean: 1.5533763343764866
Call PnL at 9:18 for strike: 4, Total: 276877.5, Per Day: 1036.9943820224719, Days: 267, Pc Mean: 1.2849965646637946
Call PnL at 9:18 for strike: 5, Total: 149872.5000000001, Per Day: 724.0217391304352, Days: 207, Pc Mean: 0.7762541829311271
Call PnL at 9:18 for strike: 6, Total: 154740.0, Per Day: 979.367088607595, Days: 158, Pc Mean: 1.2739409931218943
Call PnL at 9:18 for strike: 7, Total: 10914.999999999995, Per Day: 94.09482758620685, Days: 116, Pc Mean: 0.12636272094800613
Call PnL at 9:18 for strike: 8, Total: 9917.499999999989, Per Day: 113.9942528735631, Days: 87, Pc Mean: 0.18813370356162376
Call PnL at 9:18 for strike: 9, Total: 79105.00000000006, Per Day: 1180.6716417910457, Days: 67, Pc Mean: 1.5818898689081564
Call PnL at 9:18 for strike: 10, Total: 35062.50000000002, Per Day: 637.5000000000003, Days: 55, Pc Mean: 1.1953436107573956
Call PnL at 9:18 for strike: 11, Total: 20739.99999999998, Per Day: 460.88888888888846, Days: 45, Pc Mean: 0.7593888029924587
Call PnL at 9:18 for strike: 12, Total: -55417.49999999998, Per Day: -1288.7790697674413, Days: 43, Pc Mean: -2.0308454534870015
Call PnL at 9:18 for strike: 13, Total: 21262.50000000002, Per Day: 574.6621621621626, Days: 37, Pc Mean: 0.8487115283167249
Call PnL at 9:18 for strike: 14, Total: -24174.999999999978, Per Day: -1510.9374999999986, Days: 16, Pc Mean: -1.9692430667294567

Results Training dataset 1st expiry
Call PnL at 9:18 for strike: -10, Total: -1200977.5, Per Day: -3263.5258152173915, Days: 368, Pc Mean: -3.2645265341374787
Call PnL at 9:18 for strike: -9, Total: -1498797.5, Per Day: -4072.819293478261, Days: 368, Pc Mean: -4.07595378510088
Call PnL at 9:18 for strike: -8, Total: -1580342.5, Per Day: -4294.408967391304, Days: 368, Pc Mean: -4.300811979583259
Call PnL at 9:18 for strike: -7, Total: -1589612.5, Per Day: -4319.599184782609, Days: 368, Pc Mean: -4.33081776982772
Call PnL at 9:18 for strike: -6, Total: -1313277.5, Per Day: -3568.688858695652, Days: 368, Pc Mean: -3.581478936364814
Call PnL at 9:18 for strike: -5, Total: -957462.4999999998, Per Day: -2601.8002717391296, Days: 368, Pc Mean: -2.6178756181139833
Call PnL at 9:18 for strike: -4, Total: -376049.9999999999, Per Day: -1021.8749999999997, Days: 368, Pc Mean: -1.0172680452979237
Call PnL at 9:18 for strike: -3, Total: 861522.5, Per Day: 2341.09375, Days: 368, Pc Mean: 2.3639857872910177
Call PnL at 9:18 for strike: -2, Total: 1831165.0, Per Day: 4975.991847826087, Days: 368, Pc Mean: 5.016709407810276
Call PnL at 9:18 for strike: -1, Total: 1928680.0, Per Day: 5240.978260869565, Days: 368, Pc Mean: 5.2977612436011565
Call PnL at 9:18 for strike: 0, Total: 1511580.0, Per Day: 4107.554347826087, Days: 368, Pc Mean: 4.227216010633293
Call PnL at 9:18 for strike: 1, Total: 1021664.9999999998, Per Day: 2776.263586956521, Days: 368, Pc Mean: 3.0202142490892543
Call PnL at 9:18 for strike: 2, Total: 790220.0000000002, Per Day: 2147.33695652174, Days: 368, Pc Mean: 2.2219541615482883
Call PnL at 9:18 for strike: 3, Total: 529924.9999999999, Per Day: 1443.9373297002721, Days: 367, Pc Mean: 1.659199832293209
Call PnL at 9:18 for strike: 4, Total: 404357.50000000006, Per Day: 1117.0096685082874, Days: 362, Pc Mean: 1.2296016828922414

Results Test dataset 1st expiry
Call PnL at 9:18 for strike: -5, Total: -368580.0000000001, Per Day: -996.1621621621625, Days: 370, Pc Mean: -0.9981795406298148
Call PnL at 9:18 for strike: -4, Total: -66219.99999999988, Per Day: -178.97297297297266, Days: 370, Pc Mean: -0.18476101433523964
Call PnL at 9:18 for strike: -3, Total: 29995.000000000087, Per Day: 81.0675675675678, Days: 370, Pc Mean: 0.07554203515727084
Call PnL at 9:18 for strike: -2, Total: 574157.5, Per Day: 1551.777027027027, Days: 370, Pc Mean: 1.5985776757810657
Call PnL at 9:18 for strike: -1, Total: 827162.5000000002, Per Day: 2235.574324324325, Days: 370, Pc Mean: 2.2925136232157
Call PnL at 9:18 for strike: 0, Total: 641042.5, Per Day: 1732.5472972972973, Days: 370, Pc Mean: 1.798247708632138
Call PnL at 9:18 for strike: 1, Total: 467930.0000000001, Per Day: 1264.675675675676, Days: 370, Pc Mean: 1.3283195796282112
Call PnL at 9:18 for strike: 2, Total: 390254.99999999994, Per Day: 1054.7432432432431, Days: 370, Pc Mean: 1.0203631153733186
Call PnL at 9:18 for strike: 3, Total: 209460.00000000003, Per Day: 567.6422764227643, Days: 369, Pc Mean: 0.7726631681100194
Call PnL at 9:18 for strike: 4, Total: 245740.00000000006, Per Day: 678.839779005525, Days: 362, Pc Mean: 0.7254304369662518

Results all dataset 1st expiry
Call PnL at 9:18 for strike: -3, Total: 891517.5, Per Day: 1208.0182926829268, Days: 738, Pc Mean: 1.2166630389312796
Call PnL at 9:18 for strike: -2, Total: 2405322.5, Per Day: 3259.2445799457996, Days: 738, Pc Mean: 3.303011926982623
Call PnL at 9:18 for strike: -1, Total: 2755842.5, Per Day: 3734.2039295392956, Days: 738, Pc Mean: 3.791065282161294
"""

# for i in range(-15, 30):
# for i in range(5, 10):
# for i in range(10, 15):
# for i in range(-10, -5):
# for i in range(-5, 0):
# for i in range(0, 5):
# for i in range(-2, 4):
#     strike_key = f"cur_call_atm_strike_{i}"
#     train_dates[strike_key] = train_dates["atm_strike"].apply(lambda r: get_next_1000(r, i))
#     buy_at = f"call_at_0328_{i}"
#     sell_at = f"call_at_0918_{i}"
#     train_dates[buy_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.previous_trading_day.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=28)), axis=1)
#     train_dates[sell_at] = train_dates.loc[train_dates[buy_at].notna()].apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=18)), axis=1)
#     pnl_key = f"call_pnl_0918_{i}"
#     train_dates[pnl_key] = LOT_SIZE * LOT_QTY * (train_dates.loc[train_dates[buy_at].notna()][sell_at] - train_dates.loc[train_dates[buy_at].notna()][buy_at])
#     pnl_pc_key = f"call_pnl_pc_{i}"
#     train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (LOT_SIZE * LOT_QTY * train_dates.loc[train_dates[buy_at].notna()][buy_at])
#     print(f"Call PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}, Pc Mean: {train_dates[pnl_pc_key].mean()}")

"""
9:16 sell
C PnL at for strike: -2, Total: 388099.99999999994, Per Day: 1066.2087912087911, Days: 364, Pc Mean: 1.9051433431858822
C PnL at for strike: -1, Total: 382125.00000000023, Per Day: 1038.3831521739137, Days: 368, Pc Mean: 1.6345946497715929
C PnL at for strike: 0, Total: 632225.0, Per Day: 1741.6666666666667, Days: 363, Pc Mean: 1.9424979154415067
C PnL at for strike: 1, Total: 520100.0000000001, Per Day: 1440.7202216066485, Days: 361, Pc Mean: 1.547272494539886
C PnL at for strike: 2, Total: 961775.0, Per Day: 2828.75, Days: 340, Pc Mean: 2.1652326415524534

9:18 sell best for 1st expiry test dataset
C PnL at for strike: -4, Total: 69600.00000000003, Per Day: 189.13043478260877, Days: 368, Pc Mean: 4.367442798044304
C PnL at for strike: -3, Total: 196000.0, Per Day: 532.6086956521739, Days: 368, Pc Mean: 5.3759710447737055
C PnL at for strike: -2, Total: 351575.00000000006, Per Day: 955.3668478260871, Days: 368, Pc Mean: 5.362509917219268
C PnL at for strike: -1, Total: 528975.0000000001, Per Day: 1437.4320652173917, Days: 368, Pc Mean: 4.96888929654782
C PnL at for strike: 0, Total: 708574.9999999999, Per Day: 1925.4755434782605, Days: 368, Pc Mean: 4.432243639135149
"""

otype = OPTION_TYPE_PUT
otype = OPTION_TYPE_CALL
for i in range(-3, -2):
    strike_key = f"cur_{otype}_atm_strike_{i}"
    train_dates[strike_key] = train_dates["atm_strike"].apply(lambda r: get_next_1000(r, i))
    buy_at = f"{otype}_at_0328_{i}"
    sell_at = f"{otype}_at_0918_{i}"
    train_dates[buy_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.previous_trading_day.date(), otype, dt.time(hour=15, minute=28)), axis=1)
    train_dates[sell_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), otype, dt.time(hour=9, minute=16)), axis=1)
    pnl_key = f"{otype}_pnl_0328_{i}"
    train_dates[pnl_key] = LOT_SIZE * LOT_QTY * (train_dates.loc[train_dates[sell_at].notna()][sell_at] - train_dates.loc[train_dates[sell_at].notna()][buy_at])
    pnl_pc_key = f"{otype}_pnl_pc_{i}"
    train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (LOT_SIZE * LOT_QTY * train_dates.loc[train_dates[buy_at].notna()][buy_at])
    print(f"{otype} PnL at for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}, Pc Mean: {train_dates[pnl_pc_key].mean()}")


C PnL at for strike: -3, Total: 464924.9999999999, Per Day: 629.9796747967478, Days: 738, Pc Mean: 3.3781943755929276


In [4]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)

num = -3
pnl_key = f"{otype}_pnl_0328_{num}"
train_dates["dd"] = train_dates[pnl_key]

for idx, row in train_dates.iterrows():
    pr = train_dates.shift(1).loc[idx]
    if pd.isna(pr.atm_strike) or pd.isnull(pr.atm_strike):
        continue
    train_dates.loc[idx, "dd"] = pr.dd + row[pnl_key]

def format_float(value):
    return f'{value:.2f}'

train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, buy mean: {mdf[buy_at].mean() * LOT_SIZE * LOT_QTY}")

gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf[pnl_key].sum(), 2)}, buy mean: {mdf[buy_at].mean() * LOT_SIZE * LOT_QTY}")


pd.options.display.float_format = format_float

# Set the display.float_format option to use the formatting function
train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()][["previous_trading_day", "expiry", "nifty_at_0320", "atm_strike", f"cur_{otype}_atm_strike_{num}", f"{otype}_at_0918_{num}", f"{otype}_at_0328_{num}", pnl_key, "nifty_diff", "td_nifty_at_0918", "td_nifty_at_0328", f"{otype}_pnl_pc_{num}", "dd"]]

Month: 2021-01-31 00:00:00, profit: 8225.0, buy mean: 30811.25
Month: 2021-02-28 00:00:00, profit: 35050.0, buy mean: 39621.05263157895
Month: 2021-03-31 00:00:00, profit: 62025.0, buy mean: 45282.14285714286
Month: 2021-04-30 00:00:00, profit: -29725.0, buy mean: 37561.84210526315
Month: 2021-05-31 00:00:00, profit: 18900.0, buy mean: 32730.000000000004
Month: 2021-06-30 00:00:00, profit: -7400.0, buy mean: 19911.363636363636
Month: 2021-07-31 00:00:00, profit: -4450.0, buy mean: 15376.190476190475
Month: 2021-08-31 00:00:00, profit: 73350.0, buy mean: 14709.52380952381
Month: 2021-09-30 00:00:00, profit: 49125.0, buy mean: 22330.95238095238
Month: 2021-10-31 00:00:00, profit: 5450.0, buy mean: 34985.0
Month: 2021-11-30 00:00:00, profit: -22875.0, buy mean: 33001.31578947369
Month: 2021-12-31 00:00:00, profit: 135525.0, buy mean: 36517.39130434782
Month: 2022-01-31 00:00:00, profit: -9100.0, buy mean: 42050.0
Month: 2022-02-28 00:00:00, profit: -34150.0, buy mean: 52286.25000000001
Mo

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_C_atm_strike_-3,C_at_0918_-3,C_at_0328_-3,C_pnl_0328_-3,nifty_diff,td_nifty_at_0918,td_nifty_at_0328,C_pnl_pc_-3,dd
trade_date,,,,,,,,,,,,,
2021-01-01,2020-12-31,2021-01-07,13979.90,14000,14100,80.05,77.25,1400.00,34.65,13996.10,13968.65,3.62,1400.00
2021-01-04,2021-01-01,2021-01-07,14013.60,14000,14100,87.00,71.10,7950.00,79.70,14104.35,14015.00,22.36,9350.00
2021-01-05,2021-01-04,2021-01-07,14129.55,14150,14250,19.80,36.75,-8475.00,-72.75,14075.15,14142.45,-46.12,875.00
2021-01-06,2021-01-05,2021-01-07,14207.60,14200,14300,28.65,25.75,1450.00,13.50,14240.95,14198.65,11.26,2325.00
2021-01-07,2021-01-06,2021-01-07,14147.60,14150,14250,25.05,17.00,4025.00,96.75,14253.75,14127.70,47.35,6350.00
2021-01-08,2021-01-07,2021-01-14,14130.15,14150,14250,102.15,88.70,6725.00,100.60,14258.40,14138.10,15.16,13075.00
2021-01-11,2021-01-08,2021-01-14,14352.45,14350,14450,70.00,59.25,5375.00,76.90,14474.05,14347.95,18.14,18450.00
2021-01-12,2021-01-11,2021-01-14,14483.75,14500,14600,29.45,37.85,-4200.00,-30.90,14473.80,14486.55,-22.19,14250.00
2021-01-13,2021-01-12,2021-01-14,14559.50,14550,14650,44.30,38.20,3050.00,56.00,14639.80,14570.55,15.97,17300.00


In [24]:
"""
Results all_dates i=-1 expiry 1st

Month: 2021-01-31 00:00:00, profit: 139605.0
Month: 2021-02-28 00:00:00, profit: 71115.0
Month: 2021-03-31 00:00:00, profit: 16797.5
Month: 2021-04-30 00:00:00, profit: 126210.0
Month: 2021-05-31 00:00:00, profit: 34687.5
Month: 2021-06-30 00:00:00, profit: 47577.5
Month: 2021-07-31 00:00:00, profit: 8377.5
Month: 2021-08-31 00:00:00, profit: 432315.0
Month: 2021-09-30 00:00:00, profit: 216330.0
Month: 2021-10-31 00:00:00, profit: 334595.0
Month: 2021-11-30 00:00:00, profit: -63745.0
Month: 2021-12-31 00:00:00, profit: 391830.0
Month: 2022-01-31 00:00:00, profit: -192085.0
Month: 2022-02-28 00:00:00, profit: -70482.5
Month: 2022-03-31 00:00:00, profit: 92437.5
Month: 2022-04-30 00:00:00, profit: -108052.5
Month: 2022-05-31 00:00:00, profit: -42137.5
Month: 2022-06-30 00:00:00, profit: -240587.5
Month: 2022-07-31 00:00:00, profit: 418250.0
Month: 2022-08-31 00:00:00, profit: 83580.0
Month: 2022-09-30 00:00:00, profit: -42612.5
Month: 2022-10-31 00:00:00, profit: 324885.0
Month: 2022-11-30 00:00:00, profit: -49162.5
Month: 2022-12-31 00:00:00, profit: -125740.0
Month: 2023-01-31 00:00:00, profit: -59820.0
Month: 2023-02-28 00:00:00, profit: -134817.5
Month: 2023-03-31 00:00:00, profit: 156280.0
Month: 2023-04-30 00:00:00, profit: -48010.0
Month: 2023-05-31 00:00:00, profit: 188342.5
Month: 2023-06-30 00:00:00, profit: 104335.0
Month: 2023-07-31 00:00:00, profit: 315392.5
Month: 2023-08-31 00:00:00, profit: -118622.5
Month: 2023-09-30 00:00:00, profit: -151150.0
Month: 2023-10-31 00:00:00, profit: -229855.0
Month: 2023-11-30 00:00:00, profit: 330382.5
Month: 2023-12-31 00:00:00, profit: 599397.5
Year: 2021-12-31 00:00:00, profit: 1755695.0
Year: 2022-12-31 00:00:00, profit: 48292.5
Year: 2023-12-31 00:00:00, profit: 951855.0
"""
pnl_key = f"call_pnl_0918_{num}"
train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].min()}, max idd: {mdf["dd"].idxmin()}")

gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].min()}, max idd: {mdf["dd"].idxmin()}")


Month: 2021-01-31 00:00:00, profit: -22600.0, max dd: nan, max idd: NaT
Month: 2021-02-28 00:00:00, profit: 46600.0, max dd: nan, max idd: NaT
Month: 2021-03-31 00:00:00, profit: -55600.0, max dd: nan, max idd: NaT
Month: 2021-04-30 00:00:00, profit: 118850.0, max dd: nan, max idd: NaT
Month: 2021-05-31 00:00:00, profit: 69800.0, max dd: nan, max idd: NaT
Month: 2021-06-30 00:00:00, profit: -13625.0, max dd: nan, max idd: NaT
Month: 2021-07-31 00:00:00, profit: 55900.0, max dd: nan, max idd: NaT
Month: 2021-08-31 00:00:00, profit: 84675.0, max dd: nan, max idd: NaT
Month: 2021-09-30 00:00:00, profit: 79025.0, max dd: nan, max idd: NaT
Month: 2021-10-31 00:00:00, profit: 41075.0, max dd: nan, max idd: NaT
Month: 2021-11-30 00:00:00, profit: -53575.0, max dd: nan, max idd: NaT
Month: 2021-12-31 00:00:00, profit: 39225.0, max dd: nan, max idd: NaT
Month: 2022-01-31 00:00:00, profit: -42825.0, max dd: nan, max idd: NaT
Month: 2022-02-28 00:00:00, profit: 87750.0, max dd: nan, max idd: NaT


/tmp/ipykernel_38714/2985883155.py:48: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].min()}, max idd: {mdf["dd"].idxmin()}")
/tmp/ipykernel_38714/2985883155.py:48: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].min()}, max idd: {mdf["dd"].idxmin()}")
/tmp/ipykernel_38714/2985883155.py:48: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].min()}, max idd: {mdf["dd"].idxmin()}")
/tmp/ipykernel_38714/2985883155.py:48: Futu

In [23]:
"""
Training dataset 1st expiry, i=0
-261950.0
2021-03-31 00:00:00

Test dataset 1st expiry i=0
18250.0
2021-01-04 00:00:00

All dataset 1st expiry i=-1:
-5765.0
2021-01-05 00:00:00
"""

print(train_dates["dd"].min())
print(train_dates["dd"].idxmin())

-215725.0
2021-03-24 00:00:00


In [11]:
train_dates = train_dates.to_csv("gapup_results.csv")
# train_dates = pd.read_csv("gapup_results.csv")

## Buy puts during the market hours

In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].close

def get_next_1000(strike, i):
    divider = 100
    reminder = strike % divider
    if reminder != 0:
        return divider * (strike // divider) - i * divider
    return divider * (strike // divider) - (i+1) * divider

last_strike = None
"""
put PnL at 3:28 for strike: -2, Total: -1016200.0, Per Day: -2768.9373297002726, Days: 367, Pc Mean: -2.6088599695693726
put PnL at 3:28 for strike: -1, Total: -818350.0000000001, Per Day: -2229.8365122615805, Days: 367, Pc Mean: -2.877046907395316
put PnL at 3:28 for strike: 0, Total: -722299.9999999999, Per Day: -1962.7717391304345, Days: 368, Pc Mean: -3.1501864396409993
put PnL at 3:28 for strike: 1, Total: -597700.0, Per Day: -1624.1847826086957, Days: 368, Pc Mean: -3.382883849858163
put PnL at 3:28 for strike: 2, Total: -503849.99999999994, Per Day: -1369.157608695652, Days: 368, Pc Mean: -3.6115974299990623

Training dataset, put long
put PnL at 3:28 for strike: -9, Total: -1852500.0, Per Day: -16840.909090909092, Days: 110, Pc Mean: -4.439315720992933
put PnL at 3:28 for strike: -8, Total: -1424925.0000000002, Per Day: -9693.367346938778, Days: 147, Pc Mean: -2.863027404233216
put PnL at 3:28 for strike: -7, Total: -2490975.0, Per Day: -11479.147465437789, Days: 217, Pc Mean: -3.8930789152502503
put PnL at 3:28 for strike: -6, Total: -2475800.0, Per Day: -9413.688212927756, Days: 263, Pc Mean: -3.792994903729857
put PnL at 3:28 for strike: -5, Total: -1488850.0000000002, Per Day: -4756.709265175719, Days: 313, Pc Mean: -2.347833438133472

Training dataset call short
C PnL at 3:28 for strike: -9, Total: 166100.0, Per Day: 451.35869565217394, Days: 368, Pc Mean: 26.41369376188105
C PnL at 3:28 for strike: -8, Total: 187325.0, Per Day: 509.0353260869565, Days: 368, Pc Mean: 26.878474821106145
C PnL at 3:28 for strike: -7, Total: 199400.0, Per Day: 541.8478260869565, Days: 368, Pc Mean: 26.888533120479664
C PnL at 3:28 for strike: -6, Total: 190100.0, Per Day: 516.5760869565217, Days: 368, Pc Mean: 26.149806406662552
C PnL at 3:28 for strike: -5, Total: 143875.0, Per Day: 390.9646739130435, Days: 368, Pc Mean: 24.57488719633468

C PnL at 3:28 for strike: 4, Total: 325724.99999999977, Per Day: 1119.3298969072157, Days: 291, Pc Mean: 0.015990427098674515
C PnL at 3:28 for strike: 5, Total: 1517700.0000000002, Per Day: 6144.534412955467, Days: 247, Pc Mean: 0.08777906304222093
C PnL at 3:28 for strike: 6, Total: 885475.0000000002, Per Day: 4587.953367875649, Days: 193, Pc Mean: 0.06554219096965225
C PnL at 3:28 for strike: 7, Total: 685175.0, Per Day: 4859.397163120568, Days: 141, Pc Mean: 0.069419959473151
C PnL at 3:28 for strike: 8, Total: 272924.99999999994, Per Day: 2183.3999999999996, Days: 125, Pc Mean: 0.03119142857142857
"""
# for i in range(-15, 30):
# for i in range(5, 10):
# for i in range(10, 15):
# for i in range(-10, -5):
# for i in range(-5, 0):
# for i in range(0, 5):
otype = OPTION_TYPE_PUT
otype = OPTION_TYPE_CALL
for i in range(0, 4):
    strike_key = f"cur_{otype}_atm_strike_{i}"
    train_dates[strike_key] = train_dates["atm_strike"].apply(lambda r: get_next_1000(r, i))
    buy_at = f"{otype}_at_0918_{i}"
    sell_at = f"{otype}_at_0328_{i}"
    train_dates[sell_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), otype, dt.time(hour=9, minute=16)), axis=1)
    train_dates[buy_at] = train_dates.loc[train_dates[sell_at].notna()].apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), otype, dt.time(hour=15, minute=28)), axis=1)
    pnl_key = f"{otype}_pnl_0328_{i}"
    train_dates[pnl_key] = LOT_SIZE * LOT_QTY * (train_dates.loc[train_dates[sell_at].notna()][sell_at] - train_dates.loc[train_dates[sell_at].notna()][buy_at])
    pnl_pc_key = f"{otype}_pnl_pc_{i}"
    # train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (LOT_SIZE * LOT_QTY * train_dates.loc[train_dates[buy_at].notna()][buy_at])
    train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (LOT_SIZE * 140000)
    print(f"{otype} PnL at 3:28 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}, Pc Mean: {train_dates[pnl_pc_key].mean()}")


C PnL at 3:28 for strike: 0, Total: -183350.0000000001, Per Day: -499.5912806539512, Days: 367, Pc Mean: -0.0071370182950564465
C PnL at 3:28 for strike: 1, Total: -177474.99999999997, Per Day: -486.2328767123287, Days: 365, Pc Mean: -0.006946183953033275
C PnL at 3:28 for strike: 2, Total: -94275.0, Per Day: -266.3135593220339, Days: 354, Pc Mean: -0.0038044794188862164
C PnL at 3:28 for strike: 3, Total: 459400.0, Per Day: 1404.8929663608562, Days: 327, Pc Mean: 0.020069899519440756


In [8]:
num = 3
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)
pnl_key = f"{otype}_pnl_0328_{num}"
train_dates["dd"] = train_dates[pnl_key]

for idx, row in train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()].iterrows():
    pr = train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()].shift(1).loc[idx]
    if pd.isna(pr.atm_strike) or pd.isnull(pr.atm_strike):
        continue
    train_dates.loc[idx, "dd"] = pr.dd + row[pnl_key]

def format_float(value):
    return f'{value:.2f}'

# Set the display.float_format option to use the formatting function
pd.options.display.float_format = format_float
train_dates.style.set_sticky(axis="columns")

train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()][["previous_trading_day", "expiry", "nifty_at_0320", "atm_strike", f"cur_{otype}_atm_strike_{num}", f"{otype}_at_0918_{num}", f"{otype}_at_0328_{num}", pnl_key, "td_nifty_diff", "td_nifty_at_0918", "td_nifty_at_0328", f"{otype}_pnl_pc_{num}", "dd"]]

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_C_atm_strike_3,C_at_0918_3,C_at_0328_3,C_pnl_0328_3,td_nifty_diff,td_nifty_at_0918,td_nifty_at_0328,C_pnl_pc_3,dd
trade_date,,,,,,,,,,,,,
2021-01-05,2021-01-04,2021-01-14,14129.55,14150,13800,439.25,356.00,-41625.00,123.50,14075.15,14198.65,-0.59,-41625.00
2021-01-12,2021-01-11,2021-01-21,14483.75,14500,14100,537.30,426.25,-55525.00,96.75,14473.80,14570.55,-0.79,-97150.00
2021-01-14,2021-01-13,2021-01-21,14566.60,14550,14200,448.35,415.30,-16525.00,44.55,14550.05,14594.60,-0.24,-113675.00
2021-01-15,2021-01-14,2021-01-28,14593.15,14600,14200,387.05,499.95,56450.00,-160.85,14594.35,14433.50,0.81,-57225.00
2021-01-18,2021-01-15,2021-01-28,14441.85,14450,14100,307.95,416.45,54250.00,-223.95,14453.30,14229.35,0.78,-2975.00
2021-01-22,2021-01-21,2021-02-04,14621.60,14600,14200,364.70,502.85,69075.00,-223.70,14583.40,14359.70,0.99,66100.00
2021-01-28,2021-01-27,2021-02-04,13971.30,13950,13600,355.00,389.55,17275.00,10.65,13810.40,13821.05,0.25,83375.00
2021-02-03,2021-02-02,2021-02-11,14651.85,14650,14300,548.00,497.05,-25475.00,31.65,14754.90,14786.55,-0.36,57900.00
2021-02-04,2021-02-03,2021-02-11,14787.50,14800,14400,501.00,419.20,-40900.00,97.40,14789.05,14886.45,-0.58,17000.00


In [9]:
# 1154.999999999991 for 918 to 328
train_dates.td_nifty_diff.sum()

-3345.149999999996

In [10]:
num = 3
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)
pnl_key = f"{otype}_pnl_0328_{num}"
pnl_key = "td_nifty_diff"
train_dates["dd"] = train_dates[pnl_key]

for idx, row in train_dates[train_dates[pnl_key].notna()].iterrows():
    pr = train_dates[train_dates[pnl_key].notna()].shift(1).loc[idx]
    if pd.isna(pr.atm_strike) or pd.isnull(pr.atm_strike):
        continue
    train_dates.loc[idx, "dd"] = pr.dd + row[pnl_key]

def format_float(value):
    return f'{value:.2f}'

# Set the display.float_format option to use the formatting function
pd.options.display.float_format = format_float
train_dates.style.set_sticky(axis="columns")

train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()][["previous_trading_day", "expiry", "nifty_at_0320", "atm_strike", f"cur_{otype}_atm_strike_{num}", f"{otype}_at_0918_{num}", f"{otype}_at_0328_{num}", pnl_key, "td_nifty_diff", "td_nifty_at_0918", "td_nifty_at_0328", f"{otype}_pnl_pc_{num}", "dd"]]

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_C_atm_strike_3,C_at_0918_3,C_at_0328_3,td_nifty_diff,td_nifty_diff,td_nifty_at_0918,td_nifty_at_0328,C_pnl_pc_3,dd
trade_date,,,,,,,,,,,,,
2021-01-05,2021-01-04,2021-01-14,14129.55,14150,13800,439.25,356.00,123.50,123.50,14075.15,14198.65,-0.59,142.40
2021-01-12,2021-01-11,2021-01-21,14483.75,14500,14100,537.30,426.25,96.75,96.75,14473.80,14570.55,-0.79,251.65
2021-01-14,2021-01-13,2021-01-21,14566.60,14550,14200,448.35,415.30,44.55,44.55,14550.05,14594.60,-0.24,296.20
2021-01-15,2021-01-14,2021-01-28,14593.15,14600,14200,387.05,499.95,-160.85,-160.85,14594.35,14433.50,0.81,135.35
2021-01-18,2021-01-15,2021-01-28,14441.85,14450,14100,307.95,416.45,-223.95,-223.95,14453.30,14229.35,0.78,-88.60
2021-01-22,2021-01-21,2021-02-04,14621.60,14600,14200,364.70,502.85,-223.70,-223.70,14583.40,14359.70,0.99,-312.30
2021-01-28,2021-01-27,2021-02-04,13971.30,13950,13600,355.00,389.55,10.65,10.65,13810.40,13821.05,0.25,-301.65
2021-02-03,2021-02-02,2021-02-11,14651.85,14650,14300,548.00,497.05,31.65,31.65,14754.90,14786.55,-0.36,-270.00
2021-02-04,2021-02-03,2021-02-11,14787.50,14800,14400,501.00,419.20,97.40,97.40,14789.05,14886.45,-0.58,-172.60


In [12]:
train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].max()}, max idd: {mdf["dd"].idxmin()}")

gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].max()}, max idd: {mdf["dd"].idxmin()}")


Month: 2021-01-31 00:00:00, profit: -301.65, max dd: 296.2000000000007, max idd: 2021-01-22 00:00:00
Month: 2021-02-28 00:00:00, profit: -305.9, max dd: -47.19999999999709, max idd: 2021-02-26 00:00:00
Month: 2021-03-31 00:00:00, profit: -62.0, max dd: -278.59999999999854, max idd: 2021-03-24 00:00:00
Month: 2021-04-30 00:00:00, profit: 225.65, max dd: -213.79999999999563, max idd: 2021-04-20 00:00:00
Month: 2021-05-31 00:00:00, profit: 209.45, max dd: -234.4499999999971, max idd: 2021-05-04 00:00:00
Month: 2021-06-30 00:00:00, profit: 137.6, max dd: 79.15000000000146, max idd: 2021-06-01 00:00:00
Month: 2021-07-31 00:00:00, profit: -96.6, max dd: 20.60000000000764, max idd: 2021-07-28 00:00:00
Month: 2021-08-31 00:00:00, profit: 316.05, max dd: 122.600000000004, max idd: 2021-08-18 00:00:00
Month: 2021-09-30 00:00:00, profit: -152.1, max dd: 345.95000000000255, max idd: 2021-09-30 00:00:00
Month: 2021-10-31 00:00:00, profit: -1128.65, max dd: -33.74999999999818, max idd: 2021-10-29 00